In [7]:
import numpy as np
import torch.nn as nn
import torch

In [8]:
#simplest possible transformer implementation, except seq_len=2

x = ((torch.tensor(  np.concatenate( [np.random.randint(0,50,200), np.random.randint(50,100,200) ] ), dtype=torch.float32 ) ).unsqueeze(1)).unsqueeze(1)
x= x.reshape(200,2,1)
y = torch.tensor( np.concatenate( [np.random.randint(0,50,100), np.random.randint(50,100,100)]  ), dtype=torch.float32 ).unsqueeze(1) #dont double unsqueeze cuz we just want batch#,value, not batch#,seq_len,value. bc we take x[:,-1,:] in the forward pass of the transformer, so we kill off the seq. len dim
print(y.shape) #200 samples, each has seq len 1 and vector len 1

dataset= torch.utils.data.TensorDataset(x,y)
dataloader= torch.utils.data.DataLoader(dataset,batch_size=16)

torch.Size([200, 1])


In [9]:
class Transformer(nn.Module):
    def __init__(self, hidden_d, input_d,output_d, n_head,num_layers):
        super().__init__()
        self.embed = nn.Linear(input_d,hidden_d)
        transformer_layer = nn.TransformerEncoderLayer( d_model=hidden_d, nhead=n_head , batch_first=True)
        self.transform = nn.TransformerEncoder(transformer_layer, num_layers=num_layers)
        self.unembed = nn.Linear(hidden_d,output_d)
        self.squishificate= nn.Sigmoid()

    def forward(self, x):
        x= self.embed(x)
        x= self.transform(x)
        x=self.unembed(x)
        x=self.squishificate(x)
        return x[:,-1,:] #i guess this is slightly wasteful but whatever

In [10]:
my_transformer = Transformer(128,1,1,8,2)
optimizer=torch.optim.Adam(my_transformer.parameters(),lr=1e-4)
loss_fn = torch.nn.MSELoss()

In [11]:
my_transformer.train()
for epoch in range(100):
    for x_batch, y_batch in dataloader:
        preds= my_transformer(x_batch)
        loss = loss_fn(preds,y_batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()